In [1]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import multi_rake
import re
import string
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from multi_rake import Rake
import nltk
import csv
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
#nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize, pos_tag
import scipy.stats as stats
import statistics as sta
import numpy as np

In [2]:
BB = pd.read_csv('/Users/jiaruizhang/Desktop/reviews/coding/after sort.csv')
BB = BB.sort_index()
BB

,content,star,sku,label
0,its a bagit holds candy not included trick or ...,5,8566349,1
1,the bag would be a great deal if it was availa...,1,8566349,0
2,loved the movie and it played ok was delivered...,5,6083463,1
3,as a guy i cant say im headoverheels for dirty...,3,6083463,0
4,this movie is great for all ages shipping was ...,5,8977805,1
...,...,...,...,...
154,i bought this as a christmas gift for a family...,5,3792604,0
155,our purchase was as a christmas gift for our g...,5,3792604,1
156,this is just a quick note for anyone who was c...,4,3792604,1
157,stargate sg the entire series just amazing to ...,5,8553399,0


In [3]:
online = BB[BB.label==1] #83 rows
in_store = BB[BB.label==0] # 76 rows

In [4]:
# Create quick lambda functions to find the polarity and subjectivity of each routine
from textblob import TextBlob
pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

online['polarity'] = online['content'].apply(pol)
online['subjectivity'] = online['content'].apply(sub)
polar_online = online['polarity'].sum()/83
subje_online = online['subjectivity'].sum()/83

in_store['polarity'] = in_store['content'].apply(pol)
in_store['subjectivity'] = in_store['content'].apply(sub)
polar_store = in_store['polarity'].sum()/76
subje_store = in_store['subjectivity'].sum()/76


avg_star_online = online['star'].astype(int).sum()/83
avg_star_store = in_store['star'].astype(int).sum()/76
print(("online average star rating: {:.2f}\n"+ 
       "in store average star rating: {:.2f}\n").format(avg_star_online,avg_star_store))
# star rating: in store < online 
print(("online average polarity: {:.2f} online average subjectivity: {:.2f}\n"+ 
       "in store average polarity: {:.2f} in store average subjectivity: {:.2f} \n"
      ).format(polar_online,subje_online,polar_store,subje_store))

online average star rating: 4.58
in store average star rating: 4.49

online average polarity: 0.34 online average subjectivity: 0.55
in store average polarity: 0.34 in store average subjectivity: 0.52 



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyda

In [ ]:
stats.ttest_rel(store['norm star'], online['norm star'])

In [31]:
def zerolistmaker(n):
    listofzeros = [0] * n
    return listofzeros
# online.to_csv('/Users/jiaruizhang/Desktop/reviews/coding/online_normal.csv')
# in_store.to_csv('/Users/jiaruizhang/Desktop/reviews/coding/store_normal.csv')

In [30]:
print(len(online))

83


In [21]:
import statistics 
store_star_stdev = statistics.stdev(store['star'])
online_star_stdev = statistics.stdev(online['star'])
store_star_mean = statistics.mean(store['star'])
online_star_mean = statistics.mean(online['star'])
store['norm star'] = (store['star']-store_star_mean)/store_star_stdev
online['norm star'] = (online['star']-online_star_mean)/online_star_stdev
stats.ttest_rel(store['norm star'], online['norm star'])
# product 1: 10 review, 5 from online, 5 from store, find the mean an std dev for 5 reviews, find the mean and std
# for each 5, then 5 reviews - mean / std dev, 5 reviews - mean/std dev

# 10 online, 5 store, mean of 10, std of 10, (x-mean)/std, 10 numbers 
# 5 , 5 numbers
# average the 5 and 10

Ttest_relResult(statistic=-1.3904540125212628e-15, pvalue=0.9999999999999989)

In [22]:
import statistics 
store_sub_stdev = statistics.stdev(store['subjectivity'])
online_sub_stdev = statistics.stdev(online['subjectivity'])
store_sub_mean = statistics.mean(store['subjectivity'])
online_sub_mean = statistics.mean(online['subjectivity'])
store['norm subjectivity'] = (store['subjectivity']-store_sub_mean)/store_sub_stdev
online['norm subjectivity'] = (online['subjectivity']-online_sub_mean)/online_sub_stdev
stats.ttest_rel(store['norm subjectivity'], online['norm subjectivity'])

Ttest_relResult(statistic=3.0746677157992427e-15, pvalue=0.9999999999999976)

In [88]:
online = pd.read_csv('/Users/jiaruizhang/Desktop/reviews/coding/online_normal.csv')
store = pd.read_csv('/Users/jiaruizhang/Desktop/reviews/coding/store_normal.csv')
online['polarity'] = online['content'].apply(pol)
online['subjectivity'] = online['content'].apply(sub)
store['polarity'] = store['content'].apply(pol)
store['subjectivity'] = store['content'].apply(sub)

In [89]:
store_sku = [8946929,8534124,8829155,5998654,7865115,
             8399146,9325606,9325544,8811045,9658199,
             8700015]
store['norm_polarity'] = np.nan
for i in store_sku:
    temp = []
    pos = []
    for j in range(len(store['polarity'])):
        if store.at[j,'sku'] == i:
            temp.append(store.at[j,'polarity'])
            pos.append(j)
    stdev = sta.stdev(temp)
    mean = sta.mean(temp)
    m = 0
    for k in pos:
        store.at[k,"norm_polarity"] = (temp[m]-mean)/stdev
        m += 1    
store['norm_polarity'].loc[(store['norm_polarity'].isnull())] = 0
store

,Unnamed: 0,content,star,sku,label,polarity,subjectivity,norm_polarity
0,1,the bag would be a great deal if it was availa...,1,8566349,0,0.262500,0.337500,0.000000
1,3,as a guy i cant say im headoverheels for dirty...,3,6083463,0,-0.029932,0.600680,0.000000
2,5,i ordered this dvd on cyber mondaybr best buy ...,5,8977805,0,0.370000,0.565000,0.000000
3,6,i will continue to order on bestbuycom if the ...,4,3873909,0,-0.075000,0.525000,0.000000
4,9,this movie was not one i would have typically ...,5,1067784,0,0.205556,0.666667,0.000000
...,...,...,...,...,...,...,...,...
71,148,this season of babylon had to be the best sea...,5,6074302,0,0.432576,0.389394,0.000000
72,151,thank goodness tnt advertised this was being r...,5,8700015,0,0.400000,0.547727,0.707107
73,152,can i just say quotits about timequot i have b...,5,8700015,0,0.052000,0.408000,-0.707107
74,154,i bought this as a christmas gift for a family...,5,3792604,0,0.514286,0.460714,0.000000


In [99]:
dic = {}
for j in range(len(store['polarity'])):
    if store.at[j,'sku'] not in dic:
        dic[store.at[j,'sku']] = store.at[j,'norm_polarity']
    else:
        dic[store.at[j,'sku']] = np.append(dic[store.at[j,'sku']],store.at[j,'norm_polarity'])            

{8566349: 0.0,
 6083463: 0.0,
 8977805: 0.0,
 3873909: 0.0,
 1067784: 0.0,
 5928106: 0.0,
 7548031: 0.0,
 9017066: 0.0,
 8199914: 0.0,
 8707642: 0.0,
 7204937: 0.0,
 9433222: 0.0,
 3142306: 0.0,
 8946929: array([-0.8675435 , -0.22617234,  1.09371584]),
 9065352: 0.0,
 5153095: 0.0,
 8534124: array([ 0.33966121, -0.94696071, -0.64595262,  1.25325212]),
 9615274: 0.0,
 3687406: 0.0,
 8684499: 0.0,
 8138936: 0.0,
 9019019: 0.0,
 3873838: 0.0,
 3751746: 0.0,
 9761535: 0.0,
 4503736: 0.0,
 7547325: 0.0,
 4826808: 0.0,
 3699043: 0.0,
 4826422: 0.0,
 7561033: 0.0,
 8829155: array([-0.70710678,  0.70710678]),
 9433945: 0.0,
 8220481: 0.0,
 5998654: array([-0.70710678,  0.70710678]),
 7865115: array([ 0.70710678, -0.70710678]),
 8399146: array([ 0.35372077,  0.77506462, -1.12878539]),
 9408928: 0.0,
 7788089: 0.0,
 5371545: 0.0,
 8539575: 0.0,
 9325606: array([ 0.70710678, -0.70710678]),
 6096681: 0.0,
 9325544: array([-0.70710678,  0.70710678]),
 9506117: 0.0,
 8953395: 0.0,
 7205767: 0.0,
 88

In [113]:
for key, value in dic.items():
    if value != 0.0:
        temp = sta.mean(value)
        dic[key] = temp
        if key == "8946929":
            print(temp)
            print(value)
        #value = [temp]
#dic

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [90]:
online_sku = [8946929,8534124,9615274,9761535,3699043,
              7561033,8399146,8539575,9325544,8953395,
              8811045,9460576,3792604]
online['norm_polarity'] = np.nan
for i in online_sku:
    temp = []
    pos = []
    for j in range(len(online['polarity'])):
        if online.at[j,'sku'] == i:
            temp.append(online.at[j,'polarity'])
            pos.append(j)
    stdev = sta.stdev(temp)
    mean = sta.mean(temp)
    m = 0
    for k in pos:
        online.at[k,"norm_polarity"] = (temp[m]-mean)/stdev
        m += 1    
online['norm_polarity'].loc[(online['norm_polarity'].isnull())] = 0
online

,Unnamed: 0,content,star,sku,label,polarity,subjectivity,norm_polarity
0,0,its a bagit holds candy not included trick or ...,5,8566349,1,0.400000,0.800000,0.000000
1,2,loved the movie and it played ok was delivered...,5,6083463,1,0.600000,0.650000,0.000000
2,4,this movie is great for all ages shipping was ...,5,8977805,1,0.500000,0.675000,0.000000
3,7,this movie seriously rocks from begining to en...,5,3873909,1,0.208462,0.784274,0.000000
4,8,excellent dvd very fast shipping from best bu...,5,1067784,1,0.552000,0.616000,0.000000
...,...,...,...,...,...,...,...,...
78,150,i ordered the series incluning discs i paid f...,1,8700015,1,-0.200000,1.000000,0.000000
79,153,less of hassle then buying from ebay and havin...,5,3792604,1,-0.166667,0.066667,-1.128145
80,155,our purchase was as a christmas gift for our g...,5,3792604,1,0.221429,0.308929,0.350844
81,156,this is just a quick note for anyone who was c...,4,3792604,1,0.333333,0.500000,0.777301


In [ ]:
online_polar = []
for i in online_sku:
    for j in range(len(online['polarity'])):
        if online.at[j,'sku'] == i:
            temp.append(online.at[j,'norm_polarity'])
    mean = sta.mean(temp)
    for k in pos:
        online.at[k,"norm_polarity"] = (temp[m]-mean)/stdev
        m += 1 
if 

In [91]:
stats.ttest_rel(store['norm_polarity'], online['norm_polarity'])

ValueError: unequal length arrays

In [13]:
online = pd.read_csv('/Users/jiaruizhang/Desktop/reviews/coding/online t test no dup.csv')
store = pd.read_csv('/Users/jiaruizhang/Desktop/reviews/coding/in_store t test.csv')
stats.ttest_rel(store['star'], online['star'])

Ttest_relResult(statistic=0.007537455255355957, pvalue=0.9940110436035877)

In [14]:
sta.mean(store['star'])

4.4576502732295085

In [15]:
sta.mean(online['star'])

4.4562841530163935